# A full business solution

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# # Initialize and constants

# load_dotenv(override=True)
# api_key = os.getenv('OPENAI_API_KEY')

# if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
#     print("API key looks good so far")
# else:
#     print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
# MODEL = 'gpt-4o-mini'
# openai = OpenAI()

In [3]:
MODEL = "llama3.2"
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-soft

## First step: Have Llama model figure out which links are relevant

### Use a call to Llama model to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!


In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https:/

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content

    try:
        parsed_result = json.loads(result)
        if "links" not in parsed_result:
            print("Error: 'links' key missing in response", parsed_result)
            return {"links": []}  # Return empty list if missing
        return parsed_result
    except json.JSONDecodeError as e:
        print("JSON decoding error:", e)
        print("Raw response:", result)
        return {"links": []}  # Fallback to avoid breaking execution


In [11]:
eddonner = Website("https://edwarddonner.com")
eddonner.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-soft

In [12]:
get_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'outsmart link', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'careers page', 'url': 'https://edwarddonner.com/'},
  {'type': 'posts page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'LinkedIn profile',
   'url': 'https://www.linkedin.com/in/eddonner/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)

    # Ensure "links" key exists
    if "links" not in links or not isinstance(links["links"], list):
        print("Warning: No valid links found.")
        return result

    for link in links["links"]:
        if "type" in link and "url" in link:
            result += f"\n\n{link['type']}\n"
            result += Website(link["url"]).get_contents()
        else:
            print("Skipping invalid link entry:", link)
    return result

In [14]:
print(get_all_details("https://edwarddonner.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'company page', 'url': 'https://edwarddonner.com/'}]}
Landing page:
Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acq

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("EdDonner", "https://edwarddonner.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'careers/page not found, however a general company page exists', 'url': 'https://edwarddonner.com/'}]}


'You are looking at a company called: EdDonner\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome - Edward Donner\nWebpage Contents:\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the found

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("EdDonner", "https://edwarddonner.com")

Found links: {'links': [{'type': 'about me', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'outsmart', 'url': 'https://edwarddonner.com/outsmart/'}, {'type': 'careers page is not found but careers link will come under resources', 'url': 'https://edwarddonner.com/posts/'}, {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'}, {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'}]}


**EdDonner Brochure**
=====================================

**About Us**

At EdDonner, we're on a mission to empower people to discover their potential and pursue their reason for being. We achieve this by leveraging cutting-edge AI technologies to match individuals with roles where they'll be most fulfilled and successful.

**Our Story**

Founded by co-founder and CTO, Ed Donner, our journey began in 2013 as an AI startup called untapt. We built talent marketplaces and data science software for recruitment firms, focusing on tech roles in financial services. Our innovative work caught the attention of prestigious organizations, including Accenture FinTech Innovation Lab and American Banker.

After a successful 20-year career in Financial Services, we were acquired by GQR's parent company in 2021, marking the birth of Nebula.io. Today, we're driven to tackle tangible real-world problems using our proprietary LLM-based matching model.

**Our Focus**

We work with groundbreaking, patented LLMs verticalized for talent acquisition and management. Our award-winning platform has happy customers and tons of press coverage. We aim to help people find their dream jobs, motivated by the concept of Ikigai.

**Join Our Team**

Are you passionate about AI, innovation, and making a positive impact? Do you want to join a dynamic team of like-minded individuals who share our vision? Explore our careers page to discover new opportunities.

**Get in Touch**

Stay updated with our news, insights, and perspectives on the future of Work and Talent acquisition. Follow us:

* LinkedIn: [linked-in-profile]
* Twitter: [twitter-handle]
* Facebook: [facebook-page]

Subscribe to our newsletter today!

### We're Here to Help

Find your Ikigai
Discover your potential
Unlock human prosperity

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
stream_brochure("EdDonner", "https://edwarddonner.com")

Found links: {'links': [{'type': 'company homepage', 'url': 'https://edwarddonner.com/'}, {'type': 'About page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'Blog posts', 'url': 'https://edwarddonner.com/posts/'}]}


# EdDonner Brochure

## About Us

EdDonner is an AI-powered platform that leverages Generative AI and machine learning to revolutionize the talent acquisition industry. Our patented matching model streamlines the hiring process, matching candidates with roles with unprecedented accuracy and speed.

## Mission

Our mission is to help people discover their potential and pursue their reason for being. We strive to create a more fulfilled and prosperous world by helping individuals find roles where they will be most engaged and successful.

## Founding Story

EdDonner was founded by Ed, the co-founder and CTO of our platform. With over a decade of experience in AI and recruitment, Ed is passionate about applying cutting-edge technology to real-world problems. His journey began with the founding of untapt, an AI startup acquired in 2021.

## Technology

* **Patented Matching Model**: Our award-winning algorithm matches candidates with roles without relying on keywords, providing a more accurate and efficient hiring process.
* **Generative AI**: We harness the power of Generative AI to unlock unparalleled talent acquisition capabilities.
* **Ikigai**: Our long-term goal is to help people discover their purpose in life, motivating individuals to pursue fulfilling roles that combine their passions with skills.

## Culture

At EdDonner, we foster a culture of innovation, creativity, and collaboration. Our team is passionate about applying AI to real-world problems, and we're committed to nurturing talent and driving growth.

## Customers

We work with groundbreaking recruiters who use our platform to source, understand, engage, and manage talent.

## Community

Stay connected with us:

* Follow me on LinkedIn: [link]
* Join the conversation on Twitter: [link]
* Subscribe to our newsletter: [link]

Ready to unlock your team's potential? Explore our platform today!

In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for your website:

stream_brochure("EdDonner", "https://edwarddonner.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'company homepage', 'url': 'https://edwarddonner.com/'}, {'type': 'careers page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}


# EdDonner Brochure
## About Us

At EdDonner, we're on a mission to revolutionize the way people discover their potential and pursue their reason for being. As a cutting-edge AI startup, we're dedicated to helping recruiters source, understand, engage, and manage talent using our patented Generative AI technology.

### Our Story

Founded by co-Founder and CTO Ed in 2013 as untapt, an AI-powered talent marketplace, we've always been driven by a passion for tackling real-world problems. After being selected to be part of the Accenture FinTech Innovation Lab and receiving coverage from top publications like Fast Company, Forbes, and American Banker, our team saw exponential growth.

In 2021, untapt was acquired by GQR's parent company, paving the way for EdDonner to emerge as a leading force in talent management. Our mission remains unchanged: to help people find roles where they'll be most fulfilled and successful, raising human prosperity in the process.

### What We Do

*   **Generative AI Technology**: Our patented match model ensures accurate and efficient placement of candidates into roles.
*   **Ikigai-based Approach**: We strive to find individuals their dream jobs based on this Japanese concept of finding purpose and fulfillment.

## Culture

At EdDonner, we value:

*   **Collaboration**: Sharing knowledge and expertise to drive innovation and growth
*   **Networking**: Staying connected with the industry and nurturing meaningful relationships
*   **Experiential Learning**: Encouraging hands-on exploration of AI applications
*   **Continuous Improvement**: Embracing challenges as opportunities for growth

## Careers/Jobs

Join our dynamic team in:

*   **Software Engineering**: Developing cutting-edge AI solutions
*   **Data Science**: Unlocking insights and driving success with data-driven decisions
*   **Technology Leadership**: Shaping the future of talent management

**Stay Connected**

Follow us on LinkedIn, Twitter, or Facebook to stay up-to-date on our latest innovations and mission-driven initiatives.

EdDonner is shaping the face of talent management. Join our journey and help create a brighter future for people everywhere.